In [1]:
# GOAL: 
# Given a qasm file content apply the transformation, then run the program 
# on the same platform (QISKIT), which has a perfect reading of QASM files
# and them:
# - run program VS
# - run matamorph(program)

# Metamorphic relationship 
# - 'qubit_order': permute the order of qubits but fix the mapping to HW
# - ''

In [41]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import sqlite3 as sl
import os

In [42]:
%load_ext autoreload
%autoreload 2

In [43]:
from detectors import KS_Detector

In [68]:
#QASM_PATH = "stub_files/example.qasm"
QASM_PATH = "stub_files/easy_example.qasm"
QASM_CONTENT = open(QASM_PATH, 'r').read()

In [69]:
print(QASM_CONTENT)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cx q[0], q[1];
ry(1.11) q[2];
rz(2.22) q[2];
cx q[2], q[3];
h q[3];
h q[1];
barrier q;
measure q -> c;


In [70]:
import qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile


In [71]:
import re
import uuid
from typing import Dict, Any, List
import numpy as np

def get_n_qubits(qasm_content: str) -> int:
    m = re.search("qreg q\[(\d*)", qasm_content)
    if m:
        return int(m.group(1))
    return -1

def replace_all_reg_occurrences(qasm_content: str, start: str, end: str) -> str:
    qasm_content = qasm_content.replace(f"q[{start}]", f"q[{end}]")
    qasm_content = qasm_content.replace(f"c[{start}]", f"c[{end}]")
    return qasm_content

def scramble_qubits(qasm_content: str, qubits_mapping: Dict[int, int]):
    """Swap the qubits based on order."""
    pairs = [{"start": k, "end": v} for k, v in qubits_mapping.items()]
    triplets = [ {"tmp": uuid.uuid4().hex, **p} for p in pairs]
    print("-" * 80)
    print(qubits_mapping)
    for triplet in triplets:
        qasm_content = replace_all_reg_occurrences(
            qasm_content, start=triplet["start"], end=triplet["tmp"])
    for triplet in triplets:
        qasm_content = replace_all_reg_occurrences(
            qasm_content, start=triplet["tmp"], end=triplet["end"])
    return qasm_content
        
def create_random_mapping(qasm_content: str, seed: int = None) -> Dict[int, int]:
    max_qubits = get_n_qubits(qasm_content)
    if seed: 
        np.random.seed(seed)
    start_qubits = np.arange(max_qubits)
    np.random.shuffle(start_qubits)
    end_qubits = np.arange(max_qubits)
    np.random.shuffle(end_qubits)
    return {f: s for f, s in zip(start_qubits, end_qubits)}
  
def read_str_with_mapping(bitstring: str, direct_mapping: Dict[int, int]):
    """Given a bitstring convert it to the original mapping."""
    n_bits = len(bitstring)
    return "".join([bitstring[direct_mapping[i]] for i in range(n_bits)])
    
def convert_result_to_mapping(result: Dict[str, int], qubits_mapping: Dict[int, int]):
    """Convert the result via the given mapping.
    
    because a qubit maping will make also the result scrambled thus we have
    to reverse the mapping and read the results.
    """
    return {
        read_str_with_mapping(bitstring, qubits_mapping): freq
        for bitstring, freq in result.items()
    }

read_str_with_mapping("011", {0:1, 1:2, 2:0})

'110'

In [78]:
def morph_qubit_order(qasm_path, seed: int = 42):
    """Change the qubit order"""
    content = open(qasm_path, 'r').read()
    print("-" * 80)
    print("BEFORE")
    print(content)
    random_mapping = create_random_mapping(qasm_content=content, seed=seed) 
    new_qasm = scramble_qubits(qasm_content=content, qubits_mapping=random_mapping)
    new_qasm_path = qasm_path + "_morphed"
    with open(new_qasm_path, 'w') as out_file:
        out_file.write(new_qasm)
    print("-" * 80)
    print("AFTER")
    print(new_qasm)
    print(f"Morphed version saved here: {new_qasm_path}")
    new_qc = QuantumCircuit.from_qasm_file(new_qasm_path)
    return new_qc, random_mapping

new_qc, new_mapping = morph_qubit_order(qasm_path=QASM_PATH, seed=14)

--------------------------------------------------------------------------------
BEFORE
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[0];
cx q[0], q[1];
ry(1.11) q[2];
rz(2.22) q[2];
cx q[2], q[3];
h q[3];
h q[1];
barrier q;
measure q -> c;
--------------------------------------------------------------------------------
{1: 3, 2: 0, 0: 1, 3: 2}
--------------------------------------------------------------------------------
AFTER
OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg c[4];
h q[1];
cx q[1], q[3];
ry(1.11) q[0];
rz(2.22) q[0];
cx q[0], q[2];
h q[2];
h q[3];
barrier q;
measure q -> c;
Morphed version saved here: stub_files/easy_example.qasm_morphed


In [79]:
new_qc.draw(fold=-1)

┌──────────┐┌──────────┐           ░ ┌─┐         
q_0: ┤ Ry(1.11) ├┤ Rz(2.22) ├──■────────░─┤M├─────────
     └──┬───┬───┘└──────────┘  │        ░ └╥┘┌─┐      
q_1: ───┤ H ├─────────■────────┼────────░──╫─┤M├──────
        └───┘         │      ┌─┴─┐┌───┐ ░  ║ └╥┘┌─┐   
q_2: ─────────────────┼──────┤ X ├┤ H ├─░──╫──╫─┤M├───
                    ┌─┴─┐    ├───┤└───┘ ░  ║  ║ └╥┘┌─┐
q_3: ───────────────┤ X ├────┤ H ├──────░──╫──╫──╫─┤M├
                    └───┘    └───┘      ░  ║  ║  ║ └╥┘
c: 4/══════════════════════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [80]:
qc = QuantumCircuit.from_qasm_file(QASM_PATH)

In [81]:
qc.draw(fold=-1)

┌───┐                           ░ ┌─┐         
q_0: ───┤ H ├─────────■─────────────────░─┤M├─────────
        └───┘       ┌─┴─┐    ┌───┐      ░ └╥┘┌─┐      
q_1: ───────────────┤ X ├────┤ H ├──────░──╫─┤M├──────
     ┌──────────┐┌──┴───┴───┐└───┘      ░  ║ └╥┘┌─┐   
q_2: ┤ Ry(1.11) ├┤ Rz(2.22) ├──■────────░──╫──╫─┤M├───
     └──────────┘└──────────┘┌─┴─┐┌───┐ ░  ║  ║ └╥┘┌─┐
q_3: ────────────────────────┤ X ├┤ H ├─░──╫──╫──╫─┤M├
                             └───┘└───┘ ░  ║  ║  ║ └╥┘
c: 4/══════════════════════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [106]:
def execute_circuit(circuit_to_execute: QuantumCircuit, shots: int = 8192):
    # Transpile for simulator
    simulator = Aer.get_backend('aer_simulator')
    circ = transpile(circuit_to_execute, simulator)
    print(circ)
    # Run and get counts
    result = simulator.run(circ, shots=shots).result()
    counts = result.get_counts(circ)
    return counts

In [107]:
original_res = execute_circuit(circuit_to_execute=qc)
morphed_res = execute_circuit(circuit_to_execute=new_qc)
reordered_morphed_res = convert_result_to_mapping(morphed_res, new_mapping)

global phase: 5.1732
            ┌───┐                  ░ ┌─┐         
q_0: ───────┤ H ├─────────■────────░─┤M├─────────
            └───┘       ┌─┴─┐┌───┐ ░ └╥┘┌─┐      
q_1: ───────────────────┤ X ├┤ H ├─░──╫─┤M├──────
     ┌─────────────────┐└───┘└───┘ ░  ║ └╥┘┌─┐   
q_2: ┤ U3(1.11,2.22,0) ├──■────────░──╫──╫─┤M├───
     └─────────────────┘┌─┴─┐┌───┐ ░  ║  ║ └╥┘┌─┐
q_3: ───────────────────┤ X ├┤ H ├─░──╫──╫──╫─┤M├
                        └───┘└───┘ ░  ║  ║  ║ └╥┘
c: 4/═════════════════════════════════╩══╩══╩══╩═
                                      0  1  2  3 
global phase: 5.1732
     ┌─────────────────┐                ░ ┌─┐         
q_0: ┤ U3(1.11,2.22,0) ├──■─────────────░─┤M├─────────
     └──────┬───┬──────┘  │             ░ └╥┘┌─┐      
q_1: ───────┤ H ├─────────┼────■────────░──╫─┤M├──────
            └───┘       ┌─┴─┐  │  ┌───┐ ░  ║ └╥┘┌─┐   
q_2: ───────────────────┤ X ├──┼──┤ H ├─░──╫──╫─┤M├───
                        └───┘┌─┴─┐├───┤ ░  ║  ║ └╥┘┌─┐
q_3: ──────────────────

In [91]:
original_res

{'1001': 698,
 '0001': 707,
 '1000': 740,
 '0000': 726,
 '0111': 265,
 '0011': 719,
 '1111': 278,
 '1010': 763,
 '0110': 306,
 '0100': 286,
 '1011': 796,
 '0010': 734,
 '1101': 284,
 '0101': 319,
 '1100': 275,
 '1110': 296}

In [92]:
morphed_res

{'1101': 281,
 '1000': 739,
 '1001': 279,
 '1110': 797,
 '0111': 288,
 '0000': 684,
 '0011': 313,
 '0010': 681,
 '1100': 746,
 '0101': 274,
 '0001': 283,
 '0100': 753,
 '0110': 744,
 '1011': 288,
 '1111': 293,
 '1010': 749}

In [93]:
reordered_morphed_res

{'1110': 281,
 '0010': 739,
 '0110': 279,
 '1011': 797,
 '1101': 288,
 '0000': 684,
 '0101': 313,
 '0001': 681,
 '1010': 746,
 '1100': 274,
 '0100': 283,
 '1000': 753,
 '1001': 744,
 '0111': 288,
 '1111': 293,
 '0011': 749}

In [99]:
detector = KS_Detector()
stat, p_val_with_scrambled = detector.check(original_res, morphed_res)
print(f"p_val_with_scrambled: {p_val_with_scrambled}")
print("very low pvalue = divergent programs")

p_val_with_scrambled: 7.054386755060708e-52
very low pvalue = divergent programs


In [98]:
detector = KS_Detector()
stat, p_val_with_reordered  = detector.check(original_res, reordered_morphed_res)
print(f"p_val_with_reordered: {p_val_with_reordered}")
print("more than 0.05 pvalue = no divergence")

p_val_with_reordered: 0.9333976435134347
more than 0.05 pvalue = no divergence


# Bank of generated programs

In [ ]:
PATH_PROGRAMS = "../data/q"

In [ ]:
def motamorphic_run